In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1RJjttCvltRK-j5XaI_Tp752cibGKRYMf", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/02_00_intro.mp3"))


In [ ]:
#@title 🎧 Code Walkthrough: Setting Up Inputs
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_06_setting_up_inputs.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# Scaled Dot-Product Attention: Computing Attention from Scratch

*Part 2 of the Vizuara series on Self-Attention from First Principles*
*Estimated time: 45 minutes*

In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 1. Why Does This Matter?

In the previous notebook, we learned how to project word embeddings into Query, Key, and Value vectors. But computing Q, K, and V is just the preparation. The real magic happens when we use them to compute **attention scores** — a mechanism that lets every word in a sentence look at every other word and decide how much to pay attention to it.

The scaled dot-product attention formula is the single most important equation in modern NLP:

$$\text{Attention}(Q, K, V) = \text{softmax}\!\left(\frac{Q K^T}{\sqrt{d_k}}\right) V$$

By the end of this notebook, you will:
- Implement every step of this formula from scratch
- Work through a complete numerical example by hand and verify it with code
- Understand *why* we scale by $\sqrt{d_k}$ (and what goes wrong if we do not)
- Visualize attention weight heatmaps on real sentences

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 2. Building Intuition

Here is the core idea of attention in three sentences:

1. **Compare**: For every word, measure how relevant every other word is by comparing the word's Query against all other words' Keys (via dot product).
2. **Normalize**: Convert these raw relevance scores into a probability distribution using softmax — so they sum to 1.
3. **Aggregate**: Take a weighted average of all words' Values, using those probabilities as weights.

The result? Each word's output is a **context-enriched representation** — it now contains information from other words, weighted by how relevant those words are.

### Think About This

Why a weighted *average* of Values rather than just picking the single most relevant word? Because in natural language, meaning is often distributed across multiple words. The word "it" might need information from both "cat" (its referent) and "tired" (its state) to be fully understood.

In [ ]:
#@title 🎧 Listen: The Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_04_the_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

The attention formula has four steps. Let us write them out explicitly.

**Step 1: Raw attention scores** — $S = QK^T$

This computes the dot product between every Query and every Key. If we have $n$ words each with $d_k$-dimensional Q and K vectors, $S$ is an $n \times n$ matrix where $S_{ij}$ measures how much word $i$ should attend to word $j$.

**What this says computationally:** Each element $S_{ij} = q_i \cdot k_j = \sum_{d=1}^{d_k} q_{i,d} \cdot k_{j,d}$. When two vectors point in similar directions, their dot product is large (high attention). When they are orthogonal, it is zero (no attention).

**Step 2: Scaling** — $S_{\text{scaled}} = S / \sqrt{d_k}$

**What this says computationally:** Divide every element of $S$ by the square root of the key dimension. This prevents the dot products from becoming too large when $d_k$ is large.

**Step 3: Softmax** — $A = \text{softmax}(S_{\text{scaled}})$

**What this says computationally:** For each row (each Query word), apply softmax so the attention weights become a probability distribution that sums to 1:

$$A_{ij} = \frac{e^{S_{ij} / \sqrt{d_k}}}{\sum_{j'=1}^{n} e^{S_{ij'} / \sqrt{d_k}}}$$

**Step 4: Weighted sum** — $\text{Output} = A \cdot V$

**What this says computationally:** For each word $i$, the output is a weighted combination of all Value vectors: $\text{output}_i = \sum_{j=1}^{n} A_{ij} \cdot v_j$. Words with higher attention weights contribute more.

In [ ]:
#@title 🎧 Transition: Build It Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_05_build_it_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 4. Let's Build It — Component by Component

### 4.1 Setting Up the Inputs

Let us use the "I love math" example from the article, with known Q, K, V matrices.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import matplotlib.pyplot as plt
import numpy as np

# The sentence
words = ["I", "love", "math"]
n = len(words)
d_k = 2  # Dimension of Q, K, V vectors

# Queries (one per word)
Q = torch.tensor([
    [1.0, 0.0],  # "I"
    [0.0, 1.0],  # "love"
    [1.0, 1.0],  # "math"
], dtype=torch.float32)

# Keys (one per word)
K = torch.tensor([
    [0.0, 1.0],  # "I"
    [1.0, 0.0],  # "love"
    [1.0, 1.0],  # "math"
], dtype=torch.float32)

# Values (one per word)
V = torch.tensor([
    [1.0, 2.0],  # "I"
    [3.0, 4.0],  # "love"
    [5.0, 6.0],  # "math"
], dtype=torch.float32)

print("Q (Queries):\n", Q)
print("\nK (Keys):\n", K)
print("\nV (Values):\n", V)

In [ ]:
#@title 🎧 Listen: Step1 Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_07_step1_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.2 Step 1: Compute Raw Attention Scores ($QK^T$)

In [ ]:
#@title 🎧 Code Walkthrough: Step1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_08_step1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Step 1: QK^T
# Q has shape (3, 2), K^T has shape (2, 3)
# Result has shape (3, 3) — one score for every word pair

raw_scores = Q @ K.T  # Matrix multiplication

print("Raw attention scores (QK^T):")
print(raw_scores)
print(f"\nShape: {raw_scores.shape} — a {n}x{n} matrix")

# Let's verify one element manually:
# Q[0] dot K[0] = [1, 0] dot [0, 1] = 0
# Q[0] dot K[1] = [1, 0] dot [1, 0] = 1
# Q[0] dot K[2] = [1, 0] dot [1, 1] = 1
print(f"\nManual check row 0:")
print(f"  Q['I'] · K['I']    = {(Q[0] * K[0]).sum().item()}")
print(f"  Q['I'] · K['love'] = {(Q[0] * K[1]).sum().item()}")
print(f"  Q['I'] · K['math'] = {(Q[0] * K[2]).sum().item()}")

In [ ]:
#@title 🎧 What to Look For: Step1 Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_09_step1_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize raw attention scores
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(raw_scores.numpy(), cmap='YlOrRd', aspect='equal')
ax.set_xticks(range(n))
ax.set_xticklabels(words, fontsize=12)
ax.set_yticks(range(n))
ax.set_yticklabels(words, fontsize=12)
ax.set_xlabel('Keys (attending TO)', fontsize=12)
ax.set_ylabel('Queries (attending FROM)', fontsize=12)
ax.set_title('Raw Attention Scores (QK^T)', fontsize=14, fontweight='bold')

for i in range(n):
    for j in range(n):
        ax.text(j, i, f'{raw_scores[i,j].item():.1f}',
                ha='center', va='center', fontsize=14, fontweight='bold',
                color='white' if raw_scores[i,j].item() > 1 else 'black')

plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Step2 Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_10_step2_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.3 Step 2: Scaling by $\sqrt{d_k}$

In [ ]:
#@title 🎧 Code Walkthrough: Step2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_11_step2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Step 2: Scale by sqrt(d_k)
scale = math.sqrt(d_k)
scaled_scores = raw_scores / scale

print(f"Scale factor: sqrt({d_k}) = {scale:.3f}")
print(f"\nScaled scores:")
print(scaled_scores)

In [ ]:
#@title 🎧 Listen: Why Scaling Matters Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_12_why_scaling_matters_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us see *why* scaling matters with a demonstration.

In [ ]:
#@title 🎧 What to Look For: Scaling Demo Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_13_scaling_demo_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Code Walkthrough: Training Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_32_training_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# WHY SCALING MATTERS: demonstration
# With large d_k, dot products get very large, making softmax nearly one-hot

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, d_k_demo in zip(axes, [2, 64, 512]):
    torch.manual_seed(42)
    # Random Q and K of dimension d_k_demo
    Q_demo = torch.randn(1, d_k_demo)
    K_demo = torch.randn(5, d_k_demo)

    # Without scaling
    scores_unscaled = (Q_demo @ K_demo.T).squeeze()
    probs_unscaled = F.softmax(scores_unscaled, dim=0)

    # With scaling
    scores_scaled = scores_unscaled / math.sqrt(d_k_demo)
    probs_scaled = F.softmax(scores_scaled, dim=0)

    x_pos = np.arange(5)
    width = 0.35
    ax.bar(x_pos - width/2, probs_unscaled.numpy(), width, label='Unscaled', color='#e74c3c', alpha=0.8)
    ax.bar(x_pos + width/2, probs_scaled.numpy(), width, label='Scaled', color='#3498db', alpha=0.8)
    ax.set_title(f'd_k = {d_k_demo}', fontsize=13, fontweight='bold')
    ax.set_ylabel('Softmax probability')
    ax.set_xlabel('Key index')
    ax.legend()
    ax.set_ylim(0, 1)

plt.suptitle('Scaling Prevents Softmax from Becoming One-Hot', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
print("Notice: without scaling, as d_k grows, softmax collapses to nearly one-hot.")
print("Scaling keeps the distribution smoother, allowing gradients to flow.")

In [ ]:
#@title 🎧 Listen: Step3 Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_14_step3_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.4 Step 3: Softmax

In [ ]:
#@title 🎧 Code Walkthrough: Step3 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_15_step3_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Step 3: Apply softmax row-wise
attention_weights = F.softmax(scaled_scores, dim=-1)

print("Attention weights (after softmax):")
print(attention_weights)
print(f"\nRow sums (should all be 1.0): {attention_weights.sum(dim=1).tolist()}")

In [ ]:
#@title 🎧 Listen: Verify Softmax Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_16_verify_softmax_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us verify one row manually:

In [ ]:
#@title 🎧 Code Walkthrough: Verify Softmax Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_17_verify_softmax_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Manual softmax for Row 1 (word "I")
row1_scaled = scaled_scores[0]  # [0, 0.707, 0.707]
exp_values = torch.exp(row1_scaled)
softmax_manual = exp_values / exp_values.sum()

print(f"Scaled scores for 'I': {row1_scaled.tolist()}")
print(f"exp() values: {exp_values.tolist()}")
print(f"Sum of exp(): {exp_values.sum().item():.3f}")
print(f"Manual softmax: {softmax_manual.tolist()}")
print(f"PyTorch softmax: {attention_weights[0].tolist()}")
print(f"Match: {torch.allclose(softmax_manual, attention_weights[0], atol=1e-4)}")

In [ ]:
#@title 🎧 Listen: Step4 Heading
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_18_step4_heading.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### 4.5 Step 4: Weighted Sum of Values

In [ ]:
#@title 🎧 Code Walkthrough: Step4 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_19_step4_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Step 4: Output = Attention weights * V
output = attention_weights @ V

print("Output (context-enriched representations):")
print(output)
print(f"\nShape: {output.shape} — one output vector per word")

In [ ]:
#@title 🎧 Listen: Verify Weighted Sum Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_20_verify_weighted_sum_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


Let us verify one output row manually:

In [ ]:
#@title 🎧 Code Walkthrough: Verify Weighted Sum Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_21_verify_weighted_sum_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Manual computation for Row 3 ("math")
a_math = attention_weights[2]  # attention weights for "math"
print(f"Attention weights for 'math': {a_math.tolist()}")
print(f"\nWeighted sum:")
print(f"  {a_math[0].item():.3f} * V['I']    = {a_math[0].item():.3f} * {V[0].tolist()} = {(a_math[0]*V[0]).tolist()}")
print(f"  {a_math[1].item():.3f} * V['love']  = {a_math[1].item():.3f} * {V[1].tolist()} = {(a_math[1]*V[1]).tolist()}")
print(f"  {a_math[2].item():.3f} * V['math']  = {a_math[2].item():.3f} * {V[2].tolist()} = {(a_math[2]*V[2]).tolist()}")
output_math_manual = a_math[0]*V[0] + a_math[1]*V[1] + a_math[2]*V[2]
print(f"\n  Sum = {output_math_manual.tolist()}")
print(f"  PyTorch output: {output[2].tolist()}")
print(f"  Match: {torch.allclose(output_math_manual, output[2], atol=1e-3)}")

In [ ]:
#@title 🎧 What to Look For: Comprehensive Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_22_comprehensive_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Comprehensive attention visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. Attention heatmap
ax = axes[0]
im = ax.imshow(attention_weights.detach().numpy(), cmap='Blues', vmin=0, vmax=0.6)
ax.set_xticks(range(n))
ax.set_xticklabels(words, fontsize=12)
ax.set_yticks(range(n))
ax.set_yticklabels(words, fontsize=12)
ax.set_xlabel('Keys (attending TO)')
ax.set_ylabel('Queries (attending FROM)')
ax.set_title('Attention Weights', fontsize=13, fontweight='bold')
for i in range(n):
    for j in range(n):
        ax.text(j, i, f'{attention_weights[i,j].item():.3f}',
                ha='center', va='center', fontsize=11, fontweight='bold')
plt.colorbar(im, ax=ax, shrink=0.8)

# 2. Values
ax = axes[1]
im2 = ax.imshow(V.numpy(), cmap='RdBu_r', aspect='auto')
ax.set_xticks(range(d_k))
ax.set_xticklabels([f'Dim {i}' for i in range(d_k)])
ax.set_yticks(range(n))
ax.set_yticklabels(words, fontsize=12)
ax.set_title('Value Vectors', fontsize=13, fontweight='bold')
for i in range(n):
    for j in range(d_k):
        ax.text(j, i, f'{V[i,j].item():.1f}', ha='center', va='center', fontsize=12, fontweight='bold')
plt.colorbar(im2, ax=ax, shrink=0.8)

# 3. Output
ax = axes[2]
im3 = ax.imshow(output.detach().numpy(), cmap='RdBu_r', aspect='auto')
ax.set_xticks(range(d_k))
ax.set_xticklabels([f'Dim {i}' for i in range(d_k)])
ax.set_yticks(range(n))
ax.set_yticklabels(words, fontsize=12)
ax.set_title('Output (Weighted Sum)', fontsize=13, fontweight='bold')
for i in range(n):
    for j in range(d_k):
        ax.text(j, i, f'{output[i,j].item():.2f}', ha='center', va='center', fontsize=12, fontweight='bold')
plt.colorbar(im3, ax=ax, shrink=0.8)

plt.suptitle('From Attention Weights to Context-Enriched Outputs', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Before You Start: Todo Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_23_todo_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 5. Your Turn

### TODO 1: Implement Scaled Dot-Product Attention from Scratch

In [ ]:
#@title 🎧 Before You Start: Todo1 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_24_todo1_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def scaled_dot_product_attention(Q, K, V):
    """
    Compute scaled dot-product attention.

    Args:
        Q: Query tensor of shape (seq_len, d_k)
        K: Key tensor of shape (seq_len, d_k)
        V: Value tensor of shape (seq_len, d_v)

    Returns:
        output: Weighted sum of V, shape (seq_len, d_v)
        attention_weights: Softmax attention weights, shape (seq_len, seq_len)
    """
    d_k = Q.shape[-1]

    # ============ TODO ============
    # Step 1: Compute raw scores: Q @ K^T
    # Step 2: Scale by sqrt(d_k)
    # Step 3: Apply softmax along the last dimension
    # Step 4: Multiply by V
    # ==============================

    scores = ???       # YOUR CODE HERE
    scaled = ???       # YOUR CODE HERE
    attn_weights = ??? # YOUR CODE HERE
    output = ???       # YOUR CODE HERE

    return output, attn_weights

In [ ]:
#@title 🎧 Code Walkthrough: Todo1 Verification
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_25_todo1_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Verification
out_test, weights_test = scaled_dot_product_attention(Q, K, V)

# Check shapes
assert out_test.shape == (3, 2), f"Expected output shape (3, 2), got {out_test.shape}"
assert weights_test.shape == (3, 3), f"Expected weights shape (3, 3), got {weights_test.shape}"

# Check that weights sum to 1 per row
row_sums = weights_test.sum(dim=-1)
assert torch.allclose(row_sums, torch.ones(3), atol=1e-5), f"Weights don't sum to 1: {row_sums}"

# Check against known output
expected_output = attention_weights @ V
assert torch.allclose(out_test, expected_output, atol=1e-3), "Output doesn't match expected values"

print("All assertions passed! Your attention implementation is correct!")

In [ ]:
#@title 🎧 Before You Start: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_26_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Add Causal Masking

In [ ]:
#@title 🎧 Before You Start: Todo2 Code
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_27_todo2_code.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def masked_scaled_dot_product_attention(Q, K, V, mask=None):
    """
    Compute scaled dot-product attention with optional causal mask.

    A causal mask prevents words from attending to future words.
    This is essential for autoregressive models like GPT.

    Args:
        Q: Query tensor of shape (seq_len, d_k)
        K: Key tensor of shape (seq_len, d_k)
        V: Value tensor of shape (seq_len, d_v)
        mask: Boolean mask of shape (seq_len, seq_len).
              True = position is allowed, False = position is masked.

    Returns:
        output: Shape (seq_len, d_v)
        attention_weights: Shape (seq_len, seq_len)
    """
    d_k = Q.shape[-1]
    scores = Q @ K.T / math.sqrt(d_k)

    # ============ TODO ============
    # If mask is provided:
    #   Replace masked positions (where mask is False) with -1e9
    #   This makes softmax assign ~0 probability to those positions
    #
    # Hint: scores = scores.masked_fill(~mask, float('-inf'))
    # ==============================

    if mask is not None:
        pass  # YOUR CODE HERE

    attn_weights = F.softmax(scores, dim=-1)
    output = attn_weights @ V
    return output, attn_weights

In [ ]:
#@title 🎧 Code Walkthrough: Todo2 Verification
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_28_todo2_verification.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Test with causal mask
# Causal mask: word i can only attend to words j <= i
causal_mask = torch.tril(torch.ones(3, 3, dtype=torch.bool))
print("Causal mask (lower triangular):")
print(causal_mask.int())

out_masked, weights_masked = masked_scaled_dot_product_attention(Q, K, V, mask=causal_mask)

print("\nMasked attention weights:")
print(weights_masked)
print("\nNotice: word 0 ('I') only attends to itself")
print("Word 1 ('love') attends to 'I' and 'love' but not 'math'")
print("Word 2 ('math') attends to all words")

# Verify masking worked
assert weights_masked[0, 1].item() < 1e-6, "Word 0 should not attend to word 1"
assert weights_masked[0, 2].item() < 1e-6, "Word 0 should not attend to word 2"
assert weights_masked[1, 2].item() < 1e-6, "Word 1 should not attend to word 2"
print("\nAll masking assertions passed!")

In [ ]:
#@title 🎧 Transition: Putting It Together
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_29_putting_it_together.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

In [ ]:
#@title 🎧 Code Walkthrough: Attention Module
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_30_attention_module.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
class ScaledDotProductAttention(nn.Module):
    """
    Complete scaled dot-product attention module.
    Supports optional causal masking.
    """
    def __init__(self):
        super().__init__()

    def forward(self, Q, K, V, mask=None):
        d_k = Q.shape[-1]

        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)

        # Apply mask if provided
        if mask is not None:
            scores = scores.masked_fill(~mask, float('-inf'))

        # Softmax normalization
        attention_weights = F.softmax(scores, dim=-1)

        # Weighted sum of values
        output = torch.matmul(attention_weights, V)

        return output, attention_weights

# Test the module
attn_module = ScaledDotProductAttention()

# Without mask
out1, w1 = attn_module(Q, K, V)
print("Without mask — attention weights:")
print(w1)

# With causal mask
out2, w2 = attn_module(Q, K, V, mask=causal_mask)
print("\nWith causal mask — attention weights:")
print(w2)

In [ ]:
#@title 🎧 Transition: Training Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_31_training_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

Let us apply self-attention to a real task: next-word prediction on a tiny corpus. We will train embeddings and QKV projections end-to-end.

In [ ]:
torch.manual_seed(42)

# Tiny corpus
vocab = ['<pad>', 'the', 'cat', 'sat', 'on', 'mat', 'dog', 'ran', 'in', 'park']
word2idx = {w: i for i, w in enumerate(vocab)}
vocab_size = len(vocab)
d_model = 16
d_k = 8

# Training data: predict last word from context
# "the cat sat" -> "on", "the dog ran" -> "in"
sequences = [
    ([1, 2, 3], 4),    # the cat sat -> on
    ([1, 6, 7], 8),    # the dog ran -> in
    ([4, 1, 5], 3),    # on the mat -> sat
    ([8, 1, 9], 7),    # in the park -> ran
]

# Model components
embedding = nn.Embedding(vocab_size, d_model)
W_q = nn.Linear(d_model, d_k, bias=False)
W_k = nn.Linear(d_model, d_k, bias=False)
W_v = nn.Linear(d_model, d_k, bias=False)
output_proj = nn.Linear(d_k, vocab_size)

params = list(embedding.parameters()) + list(W_q.parameters()) + \
         list(W_k.parameters()) + list(W_v.parameters()) + list(output_proj.parameters())
optimizer = torch.optim.Adam(params, lr=0.01)

losses = []
accuracies = []

for epoch in range(300):
    total_loss = 0
    correct = 0
    for seq, target in sequences:
        x = embedding(torch.tensor(seq))  # (3, d_model)

        Q_t = W_q(x)
        K_t = W_k(x)
        V_t = W_v(x)

        # Scaled dot-product attention
        scores = Q_t @ K_t.T / math.sqrt(d_k)
        attn = F.softmax(scores, dim=-1)
        context = attn @ V_t  # (3, d_k)

        # Use last position for prediction
        logits = output_proj(context[-1])
        loss = F.cross_entropy(logits.unsqueeze(0), torch.tensor([target]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if logits.argmax().item() == target:
            correct += 1

    losses.append(total_loss / len(sequences))
    accuracies.append(correct / len(sequences))

print(f"Final loss: {losses[-1]:.4f}")
print(f"Final accuracy: {accuracies[-1]*100:.0f}%")

In [ ]:
#@title 🎧 What to Look For: Training Plots
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_33_training_plots.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Plot training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(losses, color='#e74c3c', linewidth=2)
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss', fontsize=12)
ax1.set_title('Training Loss', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(accuracies, color='#2ecc71', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Accuracy', fontsize=12)
ax2.set_title('Training Accuracy', fontsize=14, fontweight='bold')
ax2.set_ylim(-0.05, 1.05)
ax2.grid(True, alpha=0.3)

plt.suptitle('Self-Attention Learning Next-Word Prediction', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Transition: Final Output Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_34_final_output_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
#@title 🎧 What to Look For: Final Output Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_35_final_output_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Visualize learned attention patterns for each sequence
fig, axes = plt.subplots(1, len(sequences), figsize=(5*len(sequences), 4))

with torch.no_grad():
    for idx, (seq, target) in enumerate(sequences):
        x = embedding(torch.tensor(seq))
        Q_t = W_q(x)
        K_t = W_k(x)
        scores = Q_t @ K_t.T / math.sqrt(d_k)
        attn = F.softmax(scores, dim=-1)

        ax = axes[idx]
        im = ax.imshow(attn.numpy(), cmap='Blues', vmin=0, vmax=1)
        seq_words = [vocab[i] for i in seq]
        ax.set_xticks(range(3))
        ax.set_xticklabels(seq_words, fontsize=11)
        ax.set_yticks(range(3))
        ax.set_yticklabels(seq_words, fontsize=11)

        for i in range(3):
            for j in range(3):
                ax.text(j, i, f'{attn[i,j].item():.2f}',
                        ha='center', va='center', fontsize=10,
                        color='white' if attn[i,j].item() > 0.5 else 'black')

        ax.set_title(f'{" ".join(seq_words)} -> {vocab[target]}', fontsize=12, fontweight='bold')

plt.suptitle('Learned Attention Patterns', fontsize=15, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCongratulations! You have built scaled dot-product attention from scratch!")
print("You can see how the model learns different attention patterns for different sequences.")
print("Next up: Multi-Head Attention — running multiple attention heads in parallel.")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_36_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Reflection Questions
1. Looking at the attention heatmaps, which word positions does the model attend to most when predicting the next word? Does this make intuitive sense?
2. What would happen if we removed the scaling by $\sqrt{d_k}$? Try it — comment out the scaling in the training loop and observe how training behaves.
3. How does causal masking change the information available to each position? Why is this essential for autoregressive (left-to-right) generation?

### Optional Challenges
1. Implement **dropout** on the attention weights (a common technique in Transformers). Add `nn.Dropout(p=0.1)` after softmax and observe its effect on training stability.
2. Extend the training to sequences of length 5 or more. Does the model still learn meaningful attention patterns?